In [ ]:
import feedparser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils
from dateutil import parser

In [ ]:
dbname = mongo_utils.get_database()

In [ ]:
url = 'https://www.bta.bg'

In [ ]:
manin_page = requests.get(url)
man_page_data = manin_page.text
main_page = BeautifulSoup(man_page_data, 'html.parser')

In [ ]:
for link in main_page.find_all('article', class_="news-card"):
    try :
        post_url=link.find('h3',class_='news-card__title').find('a',href=True)
        news_url = post_url['href']
        news_link=url + news_url
        page = requests.get(news_link)
        data = page.text
        soup = BeautifulSoup(data, 'html.parser')
        text = soup.find('div', class_='post__content col-sm-10').get_text()
        text = text.replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
        news_title = soup.find('h1', class_='post__title').get_text().replace('site.bta', '')
        news_date = soup.find('div', class_='post__post').get_text()
        conv_date = parser.parse(news_date)
        date_posted = conv_date.strftime("%a, %d %b %y %H:%M:%S +0300")
        news_payload = {
        "date_posted" : date_posted,
        "news_link" : news_link,
        "news_text" : text,
        "news_title" : news_title
        }
        print(news_payload)
        collection_name = dbname["bta_scrape_1"]
        collection_name.insert_many([news_payload])
    except:    
        pass